In [5]:
from IPython.display import display, HTML
from IPython.display import clear_output

display(HTML("<style>.container { width:100% !important; }</style>"))

from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

import warnings
warnings.filterwarnings('default')

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import unicodedata
import re
import seaborn as sns
import time
import numpy as np
import math
import requests
import time
import datetime
#%pip install num2words
from num2words import num2words
from ast import literal_eval

# Objectif :

L'objectif de ce notebook est de traiter les valeurs manquantes, plusieurs méthodes seront utilisées pour cela :
- Parfois on ira chercher les valeurs dans d'autres variables similaires (exemple on remplacera les notes presse et joueurs de metcritic absentes par les notes presse et joueurs par celle d'igdb.com)
- L'utilisation de dictionnaires pour les platform family par exemple
- Le remplacement par le mode
- Le remplacement par la moyenne
- Le remplacement par la mediane
- L'itération sur des sous ensembles plus cohérents et homogènes relatifs à la plateforme, ou l'année de sortie d'un jeu

**Remarque**: Il n'y a pas de valeurs aberrantes en ce qui concerne les notes presse ou joueurs, ou le nombre de critiques.

In [3]:
df = pd.read_pickle('vgchartz_games_merge_metacritic_igdb.pkl')

In [ ]:
# Nettoyage erreurs virgules
# Cette correction dans le script a été faite après s'etre rendu compte d'une erreur de traitement lors de la conversion des notes (1,632 notes par exemples avaient été remplacées par 1.632 notes au lieu de 1632 notes)
# Ne pas réexcutez, c'est désormais corrigé

'''df = pd.read_pickle('vgchartz_games_merge_metacritic_igdb.pkl')

df_meta_grouped_fixed = pd.read_pickle('games_metacritic_grouped_fixed.pkl')
df_meta_fixed = pd.read_pickle('games_metacritic_fixed.pkl')
df_not_all = df[df.platform != 'All']
df_all = df[df.platform == 'All']

df_meta_grouped_fixed.reset_index(drop = True, inplace = True)
df_meta_fixed.reset_index(drop = True, inplace = True)

df_all = pd.merge(df_all, df_meta_grouped_fixed, how='left', on=['meta_game_url', 'cleaned_name']).drop(columns=['Year', 'meta_critic_review_positive_x', 'meta_critic_review_mixed_x',
       'meta_critic_review_negative_x', 'meta_user_review_positive_x',
       'meta_user_review_negative_x', 'meta_user_review_mixed_x'])
df_not_all = pd.merge(df_not_all, df_meta_fixed, how='left', on=['meta_game_url', 'cleaned_name']).drop(columns=['Name', 'Year', 'meta_critic_review_positive_x', 'meta_critic_review_mixed_x',
       'meta_critic_review_negative_x', 'meta_user_review_positive_x',
       'meta_user_review_negative_x', 'meta_user_review_mixed_x'])

my_dict = {
 'meta_critic_review_positive_y': 'meta_critic_review_positive',
 'meta_critic_review_mixed_y': 'meta_critic_review_mixed',
 'meta_critic_review_negative_y': 'meta_critic_review_negative',
 'meta_user_review_positive_y': 'meta_user_review_positive',
 'meta_user_review_negative_y': 'meta_user_review_negative',
 'meta_user_review_mixed_y': 'meta_user_review_mixed'
}

df_all.rename(columns=my_dict, inplace=True)
df_not_all.rename(columns=my_dict, inplace=True)

df = pd.concat([df_all, df_not_all]).sort_values(by = 'rank')
df.to_pickle('vgchartz_games_merge_metacritic_igdb.pkl')'''

In [8]:
df

title platform release_date  year  \
0                              Minecraft      All   2011-11-18  2011   
1                     Grand Theft Auto V      All   2013-09-17  2013   
0                             Wii Sports      Wii   2006-11-19  2006   
2          PlayerUnknown's Battlegrounds      All   2017-12-20  2017   
1                    Mario Kart 8 Deluxe       NS   2017-04-28  2017   
...                                  ...      ...          ...   ...   
19979       MX vs. ATV Supercross Encore       PC   2015-10-27  2015   
19980  Brain Exercise with Dr. Kawashima       PC   2009-05-29  2009   
19981         Brain Training for Dummies       PC   2008-10-21  2008   
19982            Bratz: Forever Diamondz      GBA   2006-09-01  2006   
19983                 Rally Championship       GC   2002-05-31  2003   

                      publisher           developer  total_sales  japan_sales  \
0                        Mojang           Mojang AB  238000000.0          0.0   
1                Rockstar Games      Rockstar North  175000000.0          0.0   
0                      Nintendo        Nintendo EAD   82900000.0          0.0   
2              PUBG Corporation    PUBG Corporation   70000000.0          0.0   
1                      Nintendo        Nintendo EPD   52000000.0          0.0   
...                         ...                 ...          ...          ...   
19979              Nordic Games     Rainbow Studios      10000.0          0.0   
19980              Namco Bandai  Namco Tales Studio      10000.0          0.0   
19981           Electronic Arts     Electronic Arts      10000.0          0.0   
19982                       THQ     Barking Lizards      10000.0          0.0   
19983  Conspiracy Entertainment             Warthog      10000.0          0.0   

       na_sales  other_sales  ...           platform_name  \
0           0.0          0.0  ...                     NaN   
1           0.0          0.0  ...                     NaN   
0           0.0          0.0  ...                     Wii   
2           0.0          0.0  ...                     NaN   
1           0.0          0.0  ...         Nintendo Switch   
...         ...          ...  ...                     ...   
19979       0.0          0.0  ...  PC (Microsoft Windows)   
19980       0.0          0.0  ...  PC (Microsoft Windows)   
19981       0.0          0.0  ...  PC (Microsoft Windows)   
19982   10000.0          0.0  ...        Game Boy Advance   
19983   10000.0          0.0  ...       Nintendo GameCube   

       platform_generation  platform_family  platform_category  \
0                      NaN              NaN                NaN   
1                      NaN              NaN                NaN   
0                      7.0         Nintendo            console   
2                      NaN              NaN                NaN   
1                      8.0         Nintendo            console   
...                    ...              ...                ...   
19979                  NaN              NaN   operating_system   
19980                  NaN              NaN   operating_system   
19981                  NaN              NaN   operating_system   
19982                  6.0         Nintendo   portable_console   
19983                  6.0         Nintendo            console   

       meta_critic_review_positive  meta_critic_review_mixed  \
0                             33.0                       0.0   
1                            107.0                       1.0   
0                             31.0                      20.0   
2                             41.0                       5.0   
1                             93.0                       2.0   
...                            ...                       ...   
19979                          NaN                       NaN   
19980                          NaN                       NaN   
19981                          NaN                       NaN   
19982                          

In [9]:
df.dtypes

title                           object
platform                        object
release_date                    object
year                            object
publisher                       object
                                ...   
meta_critic_review_mixed       float64
meta_critic_review_negative    float64
meta_user_review_positive      float64
meta_user_review_negative      float64
meta_user_review_mixed         float64
Length: 78, dtype: object

In [10]:
df['year'] =  df['release_date'].apply(lambda x: str(x)[:4])

In [11]:
df = df[df.year != 'nan']

In [12]:
df = df[df.year != '<NA>']

In [13]:
df.year.unique()

array(['2011', '2013', '2006', '2017', '2018', '2020', '1985', '2015',
       '2012', '2008', '2010', '2009', '1996', '1989', '2019', '2007',
       '2004', '2016', '2005', '2000', '2001', '2022', '1991', '2014',
       '2002', '1990', '2003', '1994', '2021', '1999', '1997', '1992',
       '1998', '1993', '1982', '1988', '1987', '1995', '1981', '1986',
       '1983', '1978', '1980', '1984', '1977', '1979', '2023'],
      dtype=object)

In [10]:
df[df.platform_generation.isna()]

title platform release_date  year  \
0                              Minecraft      All   2011-11-18  2011   
1                     Grand Theft Auto V      All   2013-09-17  2013   
2          PlayerUnknown's Battlegrounds      All   2017-12-20  2017   
3                  Red Dead Redemption 2      All   2018-10-26  2018   
4                               Terraria      All   2011-05-16  2011   
...                                  ...      ...          ...   ...   
19976                              Trine       PC   2009-07-03  2009   
19978                             MXGP 2       PC   2016-04-08  2016   
19979       MX vs. ATV Supercross Encore       PC   2015-10-27  2015   
19980  Brain Exercise with Dr. Kawashima       PC   2009-05-29  2009   
19981         Brain Training for Dummies       PC   2008-10-21  2008   

              publisher           developer  total_sales  japan_sales  \
0                Mojang           Mojang AB  238000000.0          0.0   
1        Rockstar Games      Rockstar North  175000000.0          0.0   
2      PUBG Corporation    PUBG Corporation   70000000.0          0.0   
3        Rockstar Games    Rockstar Studios   50000000.0          0.0   
4              Re-Logic            Re-Logic   44500000.0          0.0   
...                 ...                 ...          ...          ...   
19976           Nobilis          Frozenbyte      10000.0          0.0   
19978  Milestone S.r.l.    Milestone S.r.l.      10000.0          0.0   
19979      Nordic Games     Rainbow Studios      10000.0          0.0   
19980      Namco Bandai  Namco Tales Studio      10000.0          0.0   
19981   Electronic Arts     Electronic Arts      10000.0          0.0   

       na_sales  other_sales  ...           platform_name  \
0           0.0          0.0  ...                     NaN   
1           0.0          0.0  ...                     NaN   
2           0.0          0.0  ...                     NaN   
3           0.0          0.0  ...                     NaN   
4           0.0          0.0  ...                     NaN   
...         ...          ...  ...                     ...   
19976       0.0          0.0  ...  PC (Microsoft Windows)   
19978       0.0          0.0  ...  PC (Microsoft Windows)   
19979       0.0          0.0  ...  PC (Microsoft Windows)   
19980       0.0          0.0  ...  PC (Microsoft Windows)   
19981       0.0          0.0  ...  PC (Microsoft Windows)   

       platform_generation  platform_family  platform_category  \
0                      NaN              NaN                NaN   
1                      NaN              NaN                NaN   
2                      NaN              NaN                NaN   
3                      NaN              NaN                NaN   
4                      NaN              NaN                NaN   
...                    ...              ...                ...   
19976                  NaN              NaN   operating_system   
19978                  NaN              NaN   operating_system   
19979                  NaN              NaN   operating_system   
19980                  NaN              NaN   operating_system   
19981                  NaN              NaN   operating_system   

       meta_critic_review_positive  meta_critic_review_mixed  \
0                             33.0                       0.0   
1                            107.0                       1.0   
2                             41.0                       5.0   
3                            131.0                       1.0   
4                             25.0                       4.0   
...                            ...                       ...   
19976                         43.0                       7.0   
19978                          NaN                       NaN   
19979                          NaN                       NaN   
19980                          NaN                       NaN   
19981                          NaN                       NaN   

In [11]:
for year in df.year.unique():
    for platform in df[df.year == year].platform.unique():
        platform_generation_mode = np.nan
        try:
            platform_generation_mode = df[(df.year == year) & (df.platform == platform) & (df.platform_generation.notna())].platform_generation.mode()[0]
        except:
            print('error')
            pass
        print('no error')
        df.loc[(df.year == year) & (df.platform == platform) & (df.platform_generation.isna()), 'platform_generation'] = platform_generation_mode

error
no error
error
no error
no error
no error
no error
no error
no error
no error
no error
no error
error
no error
no error
error
no error
no error
no error
no error
error
no error
no error
no error
no error
no error
no error
no error
no error
no error
no error
no error
error
no error
error
no error
no error
no error
no error
no error
no error
no error
no error
no error
error
no error
no error
error
no error
no error
no error
no error
no error
no error
no error
no error
no error
error
no error
no error
no error
error
no error
no error
no error
no error
error
no error
no error
no error
no error
no error
error
no error
error
no error
no error
no error
no error
no error
no error
no error
no error
no error
no error
no error
no error
no error
no error
error
no error
no error
error
no error
error
no error
error
no error
no error
no error
no error
no error
no error
no error
no error
no error
error
no error
no error
error
no error
error
no error
no error
no error
no error
no error
no error
n

In [12]:
for year in df[df.platform_generation.isna()].year.unique():
    try:
        platform_generation_mode = df[(df.year == year) & (df.platform_generation.notna())].platform_generation.mode()[0]
        df.loc[(df.year == year) & (df.platform_generation.isna()), 'platform_generation'] = platform_generation_mode
    except:
        pass

In [13]:
df.loc[df.year == '2023', 'platform_generation'] = df.loc[df.year == '2023', 'platform_generation'].fillna(9.0)

In [14]:
df[df.platform_family.isna()].platform.unique()

array(['All', 'PC', '2600', 'GBC', 'iOS', 'PSN', 'Int', '3DO', 'WS',
       'Amig', 'NG', 'PCE', 'OSX', 'PCFX', 'Mob', 'ApII'], dtype=object)

In [15]:
df.loc[df.platform_family.isna(), 'platform_family'] = df.loc[df.platform_family.isna(), 'platform']

In [16]:
df.platform_family.replace({
    'PC':'PC',
    '2600': 'Atari',
    'GBC': 'Nintendo',
    'PSN': 'Playstation',
    'ApII': 'Apple',
    'iOS': 'Apple',
    'OSX': 'Apple',
    'Int': 'Other',
    '3DO': 'Other',
    'WS': 'Other',
    'Amig': 'Other',
    'NG': 'Other',
    'PCE': 'Other',
    'PCFX': 'Other',
    'Mob': 'Other',
    'All': 'All'
}, inplace = True)

In [17]:
df.platform_category.unique()

array([nan, 'console', 'operating_system', 'portable_console', 'computer',
       'platform'], dtype=object)

In [18]:
df.platform_category.replace({'operating_system':'computer'}, inplace = True)
df.platform_category.replace({'platform':'console'}, inplace = True)
df.loc[df.platform == 'PSN', 'platform_category'] = 'console'
df.loc[df.platform == 'GBC', 'platform_category'] = 'portable_console'
df.loc[df.platform == 'All', 'platform_category'] = 'All'

In [19]:
df.loc[df.platform == 'All', 'platform_name'] = 'All'
df.loc[df.platform == 'GBC', 'platform_name'] = 'Game Boy Color'
df.loc[df.platform == 'PSN', 'platform_name'] = 'Playstation Network'

In [20]:
df.loc[df.meta_user_score.notna(), 'meta_user_score'] = round(df.loc[df.meta_user_score.notna(), 'meta_user_score'] * 10, 0)

In [21]:
df.loc[df.meta_user_score.isna(), 'meta_user_score'] = df[df.meta_user_score.isna()].igdb_rating
df.loc[df.meta_score.isna(), 'meta_score'] = df[df.meta_score.isna()].igdb_aggregated_rating

In [22]:
df['meta_nb_press_critics'] = df['meta_critic_review_positive'] + df['meta_critic_review_mixed'] +  df['meta_critic_review_negative']

In [23]:
df =  df[['title', 'platform', 'release_date', 'year', 'publisher', 'developer',
       'total_sales', 'japan_sales', 'na_sales', 'other_sales', 'pal_sales',
       'total_sales_calculated', 'total_sales_diff', 'rank', 'meta_score', 'meta_nb_press_critics',
       'meta_nb_user_critics', 'meta_user_score', 'meta_nb_awards',
       'meta_critic_review_positive', 'meta_critic_review_mixed',
       'meta_critic_review_negative', 'meta_user_review_positive',
       'meta_user_review_negative', 'meta_user_review_mixed', 'meta_game_url',
       'id_igdb_game', 'igdb_name', 'hypes', 'game_collection',
       'game_franchise', 'nb_platforms', 'id_platform_igdb',
       'igdb_aggregated_rating', 'igdb_aggregated_rating_count', 'igdb_rating',
       'igdb_total_rating', 'igdb_total_rating_count', 'genre_adventure',
       'genre_arcade', 'genre_card-and-board-game', 'genre_fighting',
       'genre_hack-and-slash-beat-em-up', 'genre_indie', 'genre_moba',
       'genre_music', 'genre_pinball', 'genre_platform',
       'genre_point-and-click', 'genre_puzzle', 'genre_quiz-trivia',
       'genre_racing', 'genre_real-time-strategy-rts',
       'genre_role-playing-rpg', 'genre_shooter', 'genre_simulator',
       'genre_sport', 'genre_strategy', 'genre_tactical',
       'genre_turn-based-strategy-tbs', 'genre_visual-novel',
       'mode_battle-royale', 'mode_co-operative', 'mode_mmo',
       'mode_multiplayer', 'mode_single-player', 'mode_split-screen',
       'perspective_auditory', 'perspective_bird-view_isometric',
       'perspective_first-person', 'perspective_side-view', 'perspective_text',
       'perspective_third-person', 'perspective_virtual-reality',
       'cleaned_name', 'platform_name', 'platform_generation',
       'platform_family', 'platform_category']]

In [24]:
df[df.meta_nb_user_critics.isna()]

title platform release_date  year  \
3                       Super Mario Bros.      NES   1985-09-13  1985   
9      Pokémon Red / Green / Blue Version       GB   1996-02-27  1998   
15                              Duck Hunt      NES   1985-10-15  1985   
22          Pokémon Gold / Silver Version       GB   2000-10-14  2000   
28                      Super Mario World     SNES   1991-08-13  1991   
...                                   ...      ...          ...   ...   
19978                              MXGP 2       PC   2016-04-08  2016   
19979        MX vs. ATV Supercross Encore       PC   2015-10-27  2015   
19980   Brain Exercise with Dr. Kawashima       PC   2009-05-29  2009   
19981          Brain Training for Dummies       PC   2008-10-21  2008   
19982             Bratz: Forever Diamondz      GBA   2006-09-01  2006   

              publisher           developer  total_sales  japan_sales  \
3              Nintendo        Nintendo EAD   40240000.0          0.0   
9              Nintendo          Game Freak   31380000.0          0.0   
15             Nintendo       Nintendo R&D1   28310000.0          0.0   
22             Nintendo          Game Freak   23100000.0          0.0   
28             Nintendo        Nintendo EAD   20610000.0          0.0   
...                 ...                 ...          ...          ...   
19978  Milestone S.r.l.    Milestone S.r.l.      10000.0          0.0   
19979      Nordic Games     Rainbow Studios      10000.0          0.0   
19980      Namco Bandai  Namco Tales Studio      10000.0          0.0   
19981   Electronic Arts     Electronic Arts      10000.0          0.0   
19982               THQ     Barking Lizards      10000.0          0.0   

       na_sales  other_sales  ...  perspective_first-person  \
3           0.0          0.0  ...                       0.0   
9           0.0          0.0  ...                       0.0   
15          0.0          0.0  ...                       NaN   
22          0.0          0.0  ...                       NaN   
28          0.0          0.0  ...                       NaN   
...         ...          ...  ...                       ...   
19978       0.0          0.0  ...                       NaN   
19979       0.0          0.0  ...                       1.0   
19980       0.0          0.0  ...                       NaN   
19981       0.0          0.0  ...                       NaN   
19982   10000.0          0.0  ...                       NaN   

       perspective_side-view  perspective_text  perspective_third-person  \
3                        1.0               0.0                       0.0   
9                        1.0               0.0                       0.0   
15                       NaN               NaN                       NaN   
22                       NaN               NaN                       NaN   
28                       NaN               NaN                       NaN   
...                      ...               ...                       ...   
19978                    NaN               NaN                       NaN   
19979                    0.0               0.0                       1.0   
19980                    NaN               NaN                       NaN   
19981                    NaN               NaN                       NaN   
19982                    NaN               NaN                       NaN   

       perspective_virtual-reality                      cleaned_name  \
3                              0.0                  super mario bros   
9                              0.0                       pokemon red   
15                             NaN                         duck hunt   
22                             NaN                      pokemon gold   
28                             NaN                 super mario world   
...                            ...                               ...   
19978                          NaN                            mxgp 2   
19979                          0.0  

In [28]:
for year in df.year.unique():
    for platform in df[df.year == year].platform.unique():
        df_filtered = df[(df.year == year) & (df.platform == platform)]
        try:
            coeff = (df_filtered.loc[df_filtered.meta_nb_user_critics.notna()].meta_nb_user_critics / df_filtered.loc[df_filtered.meta_nb_user_critics.notna()].total_sales).sum() / len(df_filtered.loc[df_filtered.meta_nb_user_critics.notna()])
            if math.isnan(coeff) == False:
                df.loc[(df.year == year) & (df.platform == platform) & (df.meta_nb_user_critics.isna()), 'meta_nb_user_critics'] = round(df.loc[(df.year == year) & (df.platform == platform) & (df.meta_nb_user_critics.isna()), 'total_sales'] * coeff, 0)
        except:
            pass

C:\Users\ladji\AppData\Local\Temp\ipykernel_26836\3675702372.py:5: RuntimeWarning: invalid value encountered in double_scalars
  coeff = (df_filtered.loc[df_filtered.meta_nb_user_critics.notna()].meta_nb_user_critics / df_filtered.loc[df_filtered.meta_nb_user_critics.notna()].total_sales).sum() / len(df_filtered.loc[df_filtered.meta_nb_user_critics.notna()])


In [29]:
for platform in df.platform.unique():
    df_filtered = df[(df.platform == platform)]
    try:
        coeff = (df_filtered.loc[df_filtered.meta_nb_user_critics.notna()].meta_nb_user_critics / df_filtered.loc[df_filtered.meta_nb_user_critics.notna()].total_sales).sum() / len(df_filtered.loc[df_filtered.meta_nb_user_critics.notna()])
        if math.isnan(coeff) == False:
            df.loc[(df.platform == platform) & (df.meta_nb_user_critics.isna()), 'meta_nb_user_critics'] = round(df.loc[(df.platform == platform) & (df.meta_nb_user_critics.isna()), 'total_sales'] * coeff, 0)
    except:
        pass

C:\Users\ladji\AppData\Local\Temp\ipykernel_26836\257102553.py:4: RuntimeWarning: invalid value encountered in double_scalars
  coeff = (df_filtered.loc[df_filtered.meta_nb_user_critics.notna()].meta_nb_user_critics / df_filtered.loc[df_filtered.meta_nb_user_critics.notna()].total_sales).sum() / len(df_filtered.loc[df_filtered.meta_nb_user_critics.notna()])


In [30]:
for year in df.year.unique():
    df_filtered = df[(df.year == year)]
    try:
        coeff = (df_filtered.loc[df_filtered.meta_nb_user_critics.notna()].meta_nb_user_critics / df_filtered.loc[df_filtered.meta_nb_user_critics.notna()].total_sales).sum() / len(df_filtered.loc[df_filtered.meta_nb_user_critics.notna()])
        if math.isnan(coeff) == False:
            df.loc[(df.year == year) & (df.meta_nb_user_critics.isna()), 'meta_nb_user_critics'] = round(df.loc[(df.year == year) & (df.meta_nb_user_critics.isna()), 'total_sales'] * coeff, 0)
    except:
        pass

In [26]:
df['meta_critic_review_positive_perc'] = round((df['meta_critic_review_positive'] / (df['meta_critic_review_positive'] + df['meta_critic_review_mixed'] + df['meta_critic_review_negative'])) * 100, 0)
df['meta_critic_review_mixed_perc'] = round((df['meta_critic_review_positive']  / (df['meta_critic_review_positive'] + df['meta_critic_review_mixed'] + df['meta_critic_review_negative'])) * 100, 0)
df['meta_critic_review_negative_perc'] = round((df['meta_critic_review_positive']  / (df['meta_critic_review_positive'] + df['meta_critic_review_mixed'] + df['meta_critic_review_negative'])) * 100, 0)

df['meta_user_review_positive_perc'] = round((df['meta_user_review_positive'] / (df['meta_user_review_positive'] + df['meta_user_review_mixed'] + df['meta_user_review_negative'])) * 100, 0)
df['meta_user_review_mixed_perc'] = round((df['meta_user_review_positive'] / (df['meta_user_review_positive'] + df['meta_user_review_mixed'] + df['meta_user_review_negative'])) * 100, 0)
df['meta_user_review_negative_perc'] = round((df['meta_user_review_positive']  / (df['meta_user_review_positive'] + df['meta_user_review_mixed'] + df['meta_user_review_negative'])) * 100, 0)

In [28]:
df['release_date'] = pd.to_datetime(df['release_date'])
df['release_month'] = df['release_date'].dt.month

In [30]:
df = df[['title', 'platform', 'release_date', 'release_month', 'year', 'publisher', 'developer',
       'total_sales', 'japan_sales', 'na_sales', 'other_sales', 'pal_sales',
       'total_sales_calculated', 'total_sales_diff', 'rank', 'meta_score',
       'meta_nb_press_critics', 'meta_nb_user_critics', 'meta_user_score',
       'meta_nb_awards', 'meta_critic_review_positive',
       'meta_critic_review_mixed', 'meta_critic_review_negative', 'meta_critic_review_positive_perc', 'meta_critic_review_mixed_perc',
       'meta_user_review_positive_perc',
       'meta_user_review_positive', 'meta_user_review_negative',
       'meta_user_review_mixed', 'meta_user_review_mixed_perc',
       'meta_critic_review_negative_perc', 'meta_user_review_negative_perc', 'meta_game_url', 'id_igdb_game', 'igdb_name',
       'hypes', 'game_collection', 'game_franchise', 'nb_platforms',
       'id_platform_igdb', 'igdb_aggregated_rating',
       'igdb_aggregated_rating_count', 'igdb_rating', 'igdb_total_rating',
       'igdb_total_rating_count', 'genre_adventure', 'genre_arcade',
       'genre_card-and-board-game', 'genre_fighting',
       'genre_hack-and-slash-beat-em-up', 'genre_indie', 'genre_moba',
       'genre_music', 'genre_pinball', 'genre_platform',
       'genre_point-and-click', 'genre_puzzle', 'genre_quiz-trivia',
       'genre_racing', 'genre_real-time-strategy-rts',
       'genre_role-playing-rpg', 'genre_shooter', 'genre_simulator',
       'genre_sport', 'genre_strategy', 'genre_tactical',
       'genre_turn-based-strategy-tbs', 'genre_visual-novel',
       'mode_battle-royale', 'mode_co-operative', 'mode_mmo',
       'mode_multiplayer', 'mode_single-player', 'mode_split-screen',
       'perspective_auditory', 'perspective_bird-view_isometric',
       'perspective_first-person', 'perspective_side-view', 'perspective_text',
       'perspective_third-person', 'perspective_virtual-reality',
       'cleaned_name', 'platform_name', 'platform_generation',
       'platform_family', 'platform_category']]

In [31]:
df[df.id_igdb_game.notna()].isnull().sum()

title                                  0
platform                               0
release_date                           0
year                                   0
publisher                              0
                                    ... 
meta_critic_review_mixed_perc       7516
meta_critic_review_negative_perc    7516
meta_user_review_positive_perc      7891
meta_user_review_mixed_perc         7891
meta_user_review_negative_perc      7891
Length: 85, dtype: int64

In [33]:
df[df.id_igdb_game.notna()]

title platform release_date  year  \
0                              Minecraft      All   2011-11-18  2011   
1                     Grand Theft Auto V      All   2013-09-17  2013   
0                             Wii Sports      Wii   2006-11-19  2006   
2          PlayerUnknown's Battlegrounds      All   2017-12-20  2017   
1                    Mario Kart 8 Deluxe       NS   2017-04-28  2017   
...                                  ...      ...          ...   ...   
19977                     Bound By Flame     X360   2014-05-06  2014   
19979       MX vs. ATV Supercross Encore       PC   2015-10-27  2015   
19980  Brain Exercise with Dr. Kawashima       PC   2009-05-29  2009   
19982            Bratz: Forever Diamondz      GBA   2006-09-01  2006   
19983                 Rally Championship       GC   2002-05-31  2003   

                      publisher           developer  total_sales  japan_sales  \
0                        Mojang           Mojang AB  238000000.0          0.0   
1                Rockstar Games      Rockstar North  175000000.0          0.0   
0                      Nintendo        Nintendo EAD   82900000.0          0.0   
2              PUBG Corporation    PUBG Corporation   70000000.0          0.0   
1                      Nintendo        Nintendo EPD   52000000.0          0.0   
...                         ...                 ...          ...          ...   
19977    Focus Home Interactive             Spiders      10000.0          0.0   
19979              Nordic Games     Rainbow Studios      10000.0          0.0   
19980              Namco Bandai  Namco Tales Studio      10000.0          0.0   
19982                       THQ     Barking Lizards      10000.0          0.0   
19983  Conspiracy Entertainment             Warthog      10000.0          0.0   

       na_sales  other_sales  ...           platform_name  \
0           0.0          0.0  ...                     All   
1           0.0          0.0  ...                     All   
0           0.0          0.0  ...                     Wii   
2           0.0          0.0  ...                     All   
1           0.0          0.0  ...         Nintendo Switch   
...         ...          ...  ...                     ...   
19977       0.0          0.0  ...                Xbox 360   
19979       0.0          0.0  ...  PC (Microsoft Windows)   
19980       0.0          0.0  ...  PC (Microsoft Windows)   
19982   10000.0          0.0  ...        Game Boy Advance   
19983   10000.0          0.0  ...       Nintendo GameCube   

       platform_generation  platform_family  platform_category  \
0                      7.0              All                All   
1                      7.0              All                All   
0                      7.0         Nintendo            console   
2                      8.0              All                All   
1                      8.0         Nintendo            console   
...                    ...              ...                ...   
19977                  7.0             Xbox            console   
19979                  8.0               PC           computer   
19980                  7.0               PC           computer   
19982                  6.0         Nintendo   portable_console   
19983                  6.0         Nintendo            console   

       meta_critic_review_positive_perc  meta_critic_review_mixed_perc  \
0                                 100.0                          100.0   
1                                  99.0                           99.0   
0                                  61.0                           61.0   
2                                  89.0                           89.0   
1                                  98.0                           98.0   
...                                 ...                            ...   
19977                               NaN                            NaN   
19979                               NaN                            NaN   
19980

# Remplacements des genres et perspectives

Pour remplacer les valeurs nulles, on choisit d'utiliser le mode, la médiane ou la moyenne en fonction la variable. \
Cepandant, au lieu de, par exemple, remplacer par le mode sur l'ensemble du dataframe, on décide de le faire en sous ensembles déterminés par l'année de sortie du jeu, et la platforme.\

On effectue la boucle 3x, en retirant à chaque fois un critère, dans le cas ou un mode, une moyenne ou une médiane n'est pas possible à déterminer à cause d'absence de données.
La meme démarche est faite pour :
- les colonnes booléennes (définies ci dessous)
- meta_nb_press_critics (le nb de critiques de la presse)
- ['meta_score', 'meta_user_score'] (le score presse et joueurs)

In [441]:
bool_cols = ['genre_adventure', 'genre_arcade',
       'genre_card-and-board-game', 'genre_fighting',
       'genre_hack-and-slash-beat-em-up', 'genre_indie', 'genre_moba',
       'genre_music', 'genre_pinball', 'genre_platform',
       'genre_point-and-click', 'genre_puzzle', 'genre_quiz-trivia',
       'genre_racing', 'genre_real-time-strategy-rts',
       'genre_role-playing-rpg', 'genre_shooter', 'genre_simulator',
       'genre_sport', 'genre_strategy', 'genre_tactical',
       'genre_turn-based-strategy-tbs', 'genre_visual-novel',
       'mode_battle-royale', 'mode_co-operative', 'mode_mmo',
       'mode_multiplayer', 'mode_single-player', 'mode_split-screen',
       'perspective_auditory', 'perspective_bird-view_isometric',
       'perspective_first-person', 'perspective_side-view', 'perspective_text',
       'perspective_third-person', 'perspective_virtual-reality']

for col in bool_cols:
    for year in df.year.unique():
        for platform in df[df.year == year].platform.unique():
            mode = np.nan
            try:
                mode = df[
                    (df.year == year) & (df.platform == platform) & (df[col].notna())
                ][col].mode()[0]
                
                mode = int(mode)
                df.loc[(df.year == year) & (df.platform == platform) & df[col].isna() & (df.id_igdb_game.notna()), col] = mode
                print(mode, year, platform)
            except:
                print('error')
                pass
            
for col in bool_cols:
    for platform in df.platform.unique():
        mode = np.nan
        try:
            mode = df[
                (df.platform == platform) & (df[col].notna())
            ][col].mode()[0]

            mode = int(mode)
            df.loc[(df.platform == platform) & df[col].isna() & (df.id_igdb_game.notna()), col] = mode
            print(mode, platform)
        except:
            print('error')
            pass
        
for col in bool_cols:
    for year in df.year.unique():
        mode = np.nan
        try:
            mode = df[
                (df.year == year) & (df[col].notna())
            ][col].mode()[0]

            mode = int(mode)
            df.loc[(df.year == year) & df[col].isna() & (df.id_igdb_game.notna()), col] = mode
            print(mode, year)
        except:
            print('error')
            pass

0 2011 All
0 2011 PC
0 2011 3DS
0 2011 X360
0 2011 PS3
0 2011 Wii
0 2011 DS
0 2011 PSV
1 2011 XBL
0 2011 PSP
1 2011 WiiU
0 2011 PS4
1 2011 NS
1 2011 XOne
0 2011 PS2
1 2011 PSN
1 2013 All
0 2013 PS3
1 2013 3DS
0 2013 X360
0 2013 PC
1 2013 WiiU
1 2013 PS4
0 2013 Wii
1 2013 XOne
0 2013 PSV
1 2013 XBL
1 2013 NS
0 2013 PSP
1 2013 DS
0 2013 iOS
0 2006 Wii
0 2006 DS
0 2006 PC
0 2006 All
0 2006 PS2
0 2006 PSP
0 2006 X360
0 2006 PS3
1 2006 GC
0 2006 GBA
0 2006 XB
0 2006 XBL
0 2006 DC
0 2006 XOne
0 2006 PS4
1 2017 All
1 2017 NS
1 2017 PC
1 2017 PS4
1 2017 3DS
1 2017 XOne
0 2017 WiiU
0 2017 PS3
0 2017 Wii
0 2017 PSV
0 2017 X360
1 2018 All
0 2018 NS
0 2018 PS4
1 2018 PC
0 2018 XOne
1 2018 3DS
0 2018 PSV
error
0 2018 Wii
0 2018 WiiU
0 2018 X360
0 2018 OSX
0 2020 NS
1 2020 All
0 2020 PC
1 2020 PS4
1 2020 XOne
error
error
error
error
error
0 1985 NES
0 1985 All
0 1985 PC
0 1985 WiiU
0 2015 All
0 2015 PC
0 2015 PS4
0 2015 XOne
1 2015 WiiU
1 2015 3DS
0 2015 PS3
0 2015 X360
0 2015 Wii
0 2015 PSV
error
1

0 1994 NG
0 1994 SCD
0 1994 NES
0 2021 NS
0 2021 All
0 2021 PC
0 2021 PS5
0 2021 PS2
error
0 1999 All
0 1999 PS
0 1999 N64
0 1999 GBC
0 1999 PC
0 1999 DC
error
0 1999 PS2
0 1999 GC
0 1999 3DS
0 1999 PSP
0 1999 WS
0 1999 XB
error
0 1999 GBA
0 1997 All
0 1997 PS
0 1997 N64
0 1997 PC
0 1997 GB
0 1997 SAT
1 1997 PS2
0 1997 Wii
0 1997 DC
0 1997 SNES
0 1997 NG
0 1997 3DS
0 1997 PSN
0 1997 Amig
0 1992 GB
0 1992 SNES
0 1992 GEN
0 1992 All
0 1992 NES
0 1992 PC
0 1992 SAT
0 1992 GBA
0 1992 PS
1 1992 GG
0 1992 N64
1 1992 Wii
0 1998 PC
0 1998 PS
0 1998 N64
0 1998 All
0 1998 GBC
0 1998 GB
0 1998 DC
0 1998 PSP
0 1998 SAT
0 1998 PSV
0 1998 GBA
error
0 1998 GC
0 1998 DS
0 1993 SNES
0 1993 GEN
0 1993 GB
0 1993 All
0 1993 PC
0 1993 NES
0 1993 SCD
0 1993 SAT
0 1993 GBA
0 1993 PS
0 1993 DS
0 1993 NG
0 1993 PSN
0 1993 PSP
1 1982 2600
error
0 1982 Int
1 1982 All
0 1988 NES
0 1988 All
0 1988 2600
0 1988 PC
0 1987 NES
0 1987 All
0 1987 2600
0 1987 PC
0 1995 PC
0 1995 SNES
0 1995 All
0 1995 GB
0 1995 PS
0 1995

0 2007 Wii
0 2007 All
0 2007 DS
0 2007 X360
0 2007 PS3
0 2007 PS2
0 2007 PSP
0 2007 PC
0 2007 GBA
0 2007 XBL
0 2007 XB
error
0 2007 3DS
0 2007 GC
0 2004 All
0 2004 DS
0 2004 PS2
0 2004 PC
0 2004 GBA
0 2004 XB
0 2004 GC
0 2004 Wii
0 2004 PSP
0 2004 PS
0 2004 XBL
0 2004 PSN
0 2004 PS3
0 2004 X360
0 2004 PSV
0 2004 NS
0 2016 All
0 2016 3DS
0 2016 PS4
0 2016 PC
0 2016 XOne
0 2016 iOS
0 2016 WiiU
0 2016 PS3
0 2016 NS
0 2016 Wii
0 2016 PSV
0 2016 X360
error
0 2005 DS
0 2005 All
0 2005 PS2
0 2005 PSP
0 2005 PC
0 2005 GBA
0 2005 GC
0 2005 X360
0 2005 XB
0 2005 Wii
0 2005 PS4
0 2005 PS3
0 2005 PSV
0 2005 NS
0 2005 XOne
error
0 2000 All
0 2000 GBC
0 2000 PS
0 2000 N64
0 2000 PC
0 2000 PS2
0 2000 DC
0 2000 GBA
0 2000 WS
0 2000 PSP
0 2000 GC
0 2000 XB
0 2000 3DS
0 2000 DS
0 2000 Wii
0 2001 All
0 2001 PS2
0 2001 PS
0 2001 GC
0 2001 XB
0 2001 GBC
0 2001 GBA
0 2001 PC
0 2001 N64
error
0 2001 DC
0 2001 DS
error
error
0 2001 PSP
0 2022 NS
0 2022 All
0 2022 PC
0 1991 SNES
0 1991 GEN
0 1991 All
0 1991 GB

0 2013 X360
0 2013 PC
0 2013 WiiU
0 2013 PS4
0 2013 Wii
0 2013 XOne
0 2013 PSV
0 2013 XBL
0 2013 NS
0 2013 PSP
0 2013 DS
0 2013 iOS
0 2006 Wii
0 2006 DS
0 2006 PC
0 2006 All
0 2006 PS2
0 2006 PSP
0 2006 X360
0 2006 PS3
0 2006 GC
0 2006 GBA
0 2006 XB
0 2006 XBL
0 2006 DC
0 2006 XOne
0 2006 PS4
0 2017 All
0 2017 NS
1 2017 PC
0 2017 PS4
0 2017 3DS
0 2017 XOne
0 2017 WiiU
0 2017 PS3
0 2017 Wii
0 2017 PSV
0 2017 X360
0 2018 All
0 2018 NS
0 2018 PS4
0 2018 PC
0 2018 XOne
0 2018 3DS
0 2018 PSV
error
0 2018 Wii
0 2018 WiiU
0 2018 X360
1 2018 OSX
0 2020 NS
0 2020 All
0 2020 PC
0 2020 PS4
0 2020 XOne
error
error
error
error
error
0 1985 NES
0 1985 All
1 1985 PC
0 1985 WiiU
0 2015 All
1 2015 PC
0 2015 PS4
0 2015 XOne
0 2015 WiiU
0 2015 3DS
0 2015 PS3
0 2015 X360
0 2015 Wii
0 2015 PSV
error
1 2015 NS
0 2015 PSP
0 2012 PC
0 2012 All
0 2012 X360
0 2012 PS3
0 2012 3DS
0 2012 DS
0 2012 Wii
0 2012 WiiU
0 2012 PSV
0 2012 PSP
0 2012 XBL
0 2012 PS4
0 2012 PSN
0 2012 XOne
0 2012 NS
0 2008 Wii
0 2008 All
0 

0 1994 3DO
0 1994 PS
0 1994 GBA
0 1994 SAT
0 1994 NG
0 1994 SCD
0 1994 NES
0 2021 NS
0 2021 All
0 2021 PC
0 2021 PS5
0 2021 PS2
error
0 1999 All
0 1999 PS
0 1999 N64
0 1999 GBC
0 1999 PC
0 1999 DC
error
0 1999 PS2
0 1999 GC
0 1999 3DS
0 1999 PSP
0 1999 WS
0 1999 XB
error
0 1999 GBA
0 1997 All
0 1997 PS
0 1997 N64
0 1997 PC
0 1997 GB
0 1997 SAT
0 1997 PS2
0 1997 Wii
0 1997 DC
0 1997 SNES
0 1997 NG
0 1997 3DS
0 1997 PSN
0 1997 Amig
0 1992 GB
0 1992 SNES
0 1992 GEN
0 1992 All
0 1992 NES
0 1992 PC
0 1992 SAT
0 1992 GBA
0 1992 PS
0 1992 GG
0 1992 N64
0 1992 Wii
0 1998 PC
0 1998 PS
0 1998 N64
0 1998 All
0 1998 GBC
0 1998 GB
0 1998 DC
0 1998 PSP
0 1998 SAT
0 1998 PSV
0 1998 GBA
error
0 1998 GC
0 1998 DS
0 1993 SNES
0 1993 GEN
0 1993 GB
0 1993 All
0 1993 PC
0 1993 NES
0 1993 SCD
0 1993 SAT
0 1993 GBA
0 1993 PS
0 1993 DS
0 1993 NG
0 1993 PSN
0 1993 PSP
0 1982 2600
error
0 1982 Int
0 1982 All
0 1988 NES
0 1988 All
0 1988 2600
0 1988 PC
0 1987 NES
0 1987 All
0 1987 2600
0 1987 PC
0 1995 PC
0 1995

...

0 2006 GBA
0 2006 XB
0 2006 XBL
0 2006 DC
0 2006 XOne
0 2006 PS4
0 2017 All
0 2017 NS
0 2017 PC
0 2017 PS4
0 2017 3DS
0 2017 XOne
0 2017 WiiU
0 2017 PS3
1 2017 Wii
0 2017 PSV
0 2017 X360
0 2018 All
0 2018 NS
0 2018 PS4
0 2018 PC
0 2018 XOne
0 2018 3DS
0 2018 PSV
error
1 2018 Wii
1 2018 WiiU
0 2018 X360
0 2018 OSX
0 2020 NS
0 2020 All
0 2020 PC
0 2020 PS4
0 2020 XOne
error
error
error
error
error
1 1985 NES
0 1985 All
0 1985 PC
0 1985 WiiU
0 2015 All
0 2015 PC
0 2015 PS4
0 2015 XOne
0 2015 WiiU
0 2015 3DS
0 2015 PS3
0 2015 X360
0 2015 Wii
0 2015 PSV
error
0 2015 NS
0 2015 PSP
0 2012 PC
0 2012 All
0 2012 X360
0 2012 PS3
0 2012 3DS
0 2012 DS
0 2012 Wii
0 2012 WiiU
0 2012 PSV
0 2012 PSP
1 2012 XBL
0 2012 PS4
1 2012 PSN
0 2012 XOne
0 2012 NS
0 2008 Wii
0 2008 All
0 2008 X360
0 2008 PS3
0 2008 PC
0 2008 DS
0 2008 PS2
0 2008 PSP
0 2008 XB
0 2008 3DS
0 2008 PSN
1 2008 XBL
error
0 2008 PS4
0 2010 PC
0 2010 All
0 2010 X360
0 2010 DS
0 2010 PS3
0 2010 Wii
0 2010 PSP
0 2010 PS2
0 2010 PSV
0 2010 3

error
error
0 1997 All
0 1997 PS
0 1997 N64
0 1997 PC
0 1997 GB
0 1997 SAT
0 1997 PS2
0 1997 Wii
0 1997 DC
0 1997 SNES
error
error
0 1997 PSN
0 1997 Amig
0 1992 GB
0 1992 SNES
0 1992 GEN
0 1992 All
0 1992 NES
0 1992 PC
1 1992 SAT
0 1992 GBA
0 1992 PS
0 1992 GG
error
0 1992 Wii
0 1998 PC
0 1998 PS
0 1998 N64
0 1998 All
0 1998 GBC
0 1998 GB
0 1998 DC
0 1998 PSP
0 1998 SAT
0 1998 PSV
0 1998 GBA
error
0 1998 GC
0 1998 DS
0 1993 SNES
0 1993 GEN
0 1993 GB
0 1993 All
0 1993 PC
0 1993 NES
0 1993 SCD
0 1993 SAT
0 1993 GBA
0 1993 PS
0 1993 DS
0 1993 NG
error
0 1993 PSP
0 1982 2600
error
0 1982 Int
0 1982 All
0 1988 NES
0 1988 All
error
0 1988 PC
0 1987 NES
0 1987 All
0 1987 2600
1 1987 PC
0 1995 PC
0 1995 SNES
0 1995 All
0 1995 GB
0 1995 PS
0 1995 GEN
0 1995 SAT
0 1995 PSP
0 1995 N64
0 1995 NG
error
error
error
0 1995 GBA
0 1981 2600
0 1981 Int
0 1981 All
error
0 1981 ApII
0 1986 NES
0 1986 All
0 1986 2600
error
0 1983 NES
0 1983 2600
0 1983 PC
0 1983 All
0 1983 3DS
0 1978 2600
0 1980 Int
0 1980

0 2004 PS
0 2004 XBL
0 2004 PSN
0 2004 PS3
0 2004 X360
0 2004 PSV
0 2004 NS
0 2016 All
0 2016 3DS
0 2016 PS4
0 2016 PC
0 2016 XOne
0 2016 iOS
0 2016 WiiU
0 2016 PS3
0 2016 NS
0 2016 Wii
0 2016 PSV
0 2016 X360
error
0 2005 DS
0 2005 All
0 2005 PS2
0 2005 PSP
0 2005 PC
0 2005 GBA
0 2005 GC
0 2005 X360
0 2005 XB
0 2005 Wii
0 2005 PS4
0 2005 PS3
0 2005 PSV
error
0 2005 XOne
error
0 2000 All
0 2000 GBC
0 2000 PS
0 2000 N64
0 2000 PC
0 2000 PS2
0 2000 DC
0 2000 GBA
0 2000 WS
0 2000 PSP
0 2000 GC
0 2000 XB
0 2000 3DS
0 2000 DS
0 2000 Wii
0 2001 All
0 2001 PS2
0 2001 PS
0 2001 GC
0 2001 XB
0 2001 GBC
0 2001 GBA
0 2001 PC
0 2001 N64
error
0 2001 DC
0 2001 DS
error
error
0 2001 PSP
0 2022 NS
0 2022 All
0 2022 PC
0 1991 SNES
0 1991 GEN
0 1991 All
0 1991 GB
0 1991 NES
0 1991 PC
0 1991 PS
0 1991 GBA
0 2014 PS4
0 2014 All
0 2014 3DS
0 2014 XOne
0 2014 WiiU
0 2014 PS3
0 2014 PC
0 2014 PSV
0 2014 X360
0 2014 Wii
0 2014 iOS
0 2014 NS
0 2014 PSP
0 2014 DS
0 2002 All
0 2002 PS2
0 2002 GC
0 2002 GBA
0 200

0 GB
0 DS
0 X360
0 SNES
0 PS3
0 PS4
0 3DS
0 PS2
0 GBA
0 GEN
0 N64
0 PS
0 XOne
0 WiiU
0 XB
0 PSP
0 2600
0 GC
0 GBC
0 PSV
0 iOS
0 PSN
0 DC
0 XBL
0 Int
0 SAT
0 SCD
0 PS5
0 3DO
1 WS
1 Amig
error
error
0 NG
error
0 OSX
0 GG
0 PCFX
error
0 ApII
0 All
0 Wii
0 NS
0 NES
0 PC
0 GB
0 DS
0 X360
0 SNES
0 PS3
0 PS4
0 3DS
0 PS2
0 GBA
0 GEN
0 N64
0 PS
0 XOne
0 WiiU
0 XB
0 PSP
0 2600
0 GC
0 GBC
0 PSV
0 iOS
0 PSN
0 DC
0 XBL
0 Int
0 SAT
0 SCD
0 PS5
0 3DO
0 WS
0 Amig
error
error
0 NG
error
0 OSX
0 GG
0 PCFX
error
0 ApII
0 All
0 Wii
0 NS
0 NES
0 PC
0 GB
0 DS
0 X360
0 SNES
0 PS3
0 PS4
0 3DS
0 PS2
0 GBA
0 GEN
0 N64
0 PS
0 XOne
0 WiiU
0 XB
0 PSP
0 2600
0 GC
0 GBC
0 PSV
0 iOS
0 PSN
0 DC
0 XBL
0 Int
0 SAT
0 SCD
0 PS5
0 3DO
0 WS
0 Amig
error
error
0 NG
error
0 OSX
0 GG
0 PCFX
error
0 ApII
0 All
0 Wii
0 NS
0 NES
0 PC
0 GB
0 DS
0 X360
0 SNES
0 PS3
0 PS4
0 3DS
0 PS2
0 GBA
0 GEN
0 N64
0 PS
0 XOne
0 WiiU
0 XB
0 PSP
0 2600
0 GC
0 GBC
0 PSV
0 iOS
0 PSN
0 DC
0 XBL
0 Int
0 SAT
0 SCD
0 PS5
0 3DO
0 WS
1 Amig
error
error
0 

0 2008
0 2010
0 2009
0 1996
0 1989
0 2019
0 2007
0 2004
0 2016
0 2005
0 2000
0 2001
0 2022
0 1991
0 2014
0 2002
0 1990
0 2003
0 1994
0 2021
0 1999
0 1997
0 1992
0 1998
0 1993
0 1982
0 1988
0 1987
0 1995
0 1981
0 1986
0 1983
0 1978
0 1980
0 1984
0 1977
0 1979
0 2023
0 2011
0 2013
0 2006
0 2017
0 2018
0 2020
0 1985
0 2015
0 2012
0 2008
0 2010
0 2009
0 1996
0 1989
0 2019
0 2007
0 2004
0 2016
0 2005
0 2000
0 2001
0 2022
0 1991
0 2014
0 2002
0 1990
0 2003
0 1994
0 2021
0 1999
0 1997
0 1992
0 1998
0 1993
0 1982
0 1988
0 1987
0 1995
0 1981
0 1986
0 1983
0 1978
0 1980
0 1984
0 1977
0 1979
0 2023
0 2011
0 2013
0 2006
0 2017
0 2018
0 2020
0 1985
0 2015
0 2012
0 2008
0 2010
0 2009
0 1996
0 1989
0 2019
0 2007
0 2004
0 2016
0 2005
0 2000
0 2001
0 2022
0 1991
0 2014
0 2002
0 1990
0 2003
0 1994
0 2021
0 1999
0 1997
0 1992
0 1998
0 1993
0 1982
0 1988
0 1987
0 1995
0 1981
0 1986
0 1983
0 1978
0 1980
0 1984
0 1977
0 1979
0 2023
0 2011
0 2013
0 2006
0 2017
0 2018
0 2020
0 1985
0 2015
0 2012
0 2008
0 2010

In [443]:
cols = ['meta_nb_press_critics']

for col in cols:
    for year in df.year.unique():
        for platform in df[df.year == year].platform.unique():
            median = np.nan
            try:
                median = df[
                    (df.year == year) & (df.platform == platform) & (df[col].notna())
                ][col].median()
                
                median = int(median)
                df.loc[(df.year == year) & (df.platform == platform) & df[col].isna() & (df.id_igdb_game.notna()), col] = median
                print(median, year, platform)
            except:
                print('error', median, year, platform)
                pass

for col in cols:
    for platform in df.platform.unique():
        median = np.nan
        try:
            median = df[
                (df.platform == platform) & (df[col].notna())
            ][col].median()

            median = int(median)
            df.loc[(df.platform == platform) & df[col].isna() & (df.id_igdb_game.notna()), col] = median
            print(median, platform)
        except:
            print('error', median, year, platform)
            pass
            
for col in cols:
    for year in df.year.unique():
        median = np.nan
        try:
            median = df[
                (df.year == year) & (df[col].notna())
            ][col].median()

            median = int(median)
            df.loc[(df.year == year) & df[col].isna() & (df.id_igdb_game.notna()), col] = median
            print(median, year)
        except:
            print('error', median, year, platform)
            pass

123 2011 All
22 2011 PC
23 2011 3DS
48 2011 X360
39 2011 PS3
15 2011 Wii
21 2011 DS
33 2011 PSV
25 2011 XBL
21 2011 PSP
33 2011 WiiU
19 2011 PS4
16 2011 NS
12 2011 XOne
21 2011 PS2
21 2011 PSN
69 2013 All
27 2013 PS3
47 2013 3DS
32 2013 X360
18 2013 PC
26 2013 WiiU
21 2013 PS4
19 2013 Wii
16 2013 XOne
20 2013 PSV
52 2013 XBL
13 2013 NS
9 2013 PSP
16 2013 DS
8 2013 iOS
22 2006 Wii
18 2006 DS
29 2006 PC
94 2006 All
27 2006 PS2
22 2006 PSP
50 2006 X360
32 2006 PS3
14 2006 GC
10 2006 GBA
23 2006 XB
24 2006 XBL
17 2006 DC
9 2006 XOne
14 2006 PS4
79 2017 All
18 2017 NS
19 2017 PC
32 2017 PS4
44 2017 3DS
16 2017 XOne
14 2017 WiiU
15 2017 PS3
15 2017 Wii
9 2017 PSV
15 2017 X360
78 2018 All
17 2018 NS
27 2018 PS4
18 2018 PC
15 2018 XOne
40 2018 3DS
14 2018 PSV
error nan 2018 Amig
13 2018 Wii
13 2018 WiiU
18 2018 X360
18 2018 OSX
83 2020 NS
87 2020 All
46 2020 PC
81 2020 PS4
16 2020 XOne
100 2020 PS5
error nan 2020 SAT
error nan 2020 DS
error nan 2020 PS2
error nan 2020 3DS
15 1985 NES
77 1985 A

In [444]:
num_cols = ['meta_score', 'meta_user_score']

for col in num_cols:
    for year in df.year.unique():
        for platform in df[df.year == year].platform.unique():
            mean = np.nan
            try:
                mean = df[
                    (df.year == year) & (df.platform == platform) & (df[col].notna())
                ][col].mean()
                
                mean = round(mean, 2)
                df.loc[(df.year == year) & (df.platform == platform) & df[col].isna() & (df.id_igdb_game.notna()), col] = mean
                print(mean, year, platform)
            except:
                print('error')
                pass
            
            
for col in num_cols:
    for platform in df.platform.unique():
        mean = np.nan
        try:
            mean = df[
                (df.platform == platform) & (df[col].notna())
            ][col].mean()

            mean = round(mean, 2)
            df.loc[(df.platform == platform) & df[col].isna() & (df.id_igdb_game.notna()), col] = mean
            print(mean, platform)
        except:
            print('error')
            pass
            
            
for col in num_cols:
    for year in df.year.unique():
        mean = np.nan
        try:
            mean = df[
                (df.year == year) & (df[col].notna())
            ][col].mean()

            mean = round(mean, 2)
            df.loc[(df.year == year) & df[col].isna() & (df.id_igdb_game.notna()), col] = mean
            print(mean, year)
        except:
            print('error')
            pass

79.77 2011 All
72.77 2011 PC
63.75 2011 3DS
67.25 2011 X360
69.97 2011 PS3
64.31 2011 Wii
63.4 2011 DS
70.08 2011 PSV
nan 2011 XBL
72.61 2011 PSP
82.0 2011 WiiU
79.5 2011 PS4
82.5 2011 NS
75.33 2011 XOne
69.42 2011 PS2
67.0 2011 PSN
75.83 2013 All
71.31 2013 PS3
65.32 2013 3DS
69.69 2013 X360
72.27 2013 PC
72.04 2013 WiiU
73.18 2013 PS4
71.43 2013 Wii
71.47 2013 XOne
67.79 2013 PSV
78.0 2013 XBL
74.75 2013 NS
73.0 2013 PSP
38.5 2013 DS
80.0 2013 iOS
60.27 2006 Wii
64.98 2006 DS
75.84 2006 PC
76.18 2006 All
67.54 2006 PS2
68.24 2006 PSP
69.37 2006 X360
72.08 2006 PS3
64.05 2006 GC
70.17 2006 GBA
66.79 2006 XB
81.0 2006 XBL
63.0 2006 DC
66.0 2006 XOne
55.0 2006 PS4
76.16 2017 All
71.0 2017 NS
77.28 2017 PC
71.82 2017 PS4
72.57 2017 3DS
72.32 2017 XOne
79.67 2017 WiiU
73.8 2017 PS3
71.0 2017 Wii
64.36 2017 PSV
73.33 2017 X360
76.72 2018 All
72.7 2018 NS
69.94 2018 PS4
73.91 2018 PC
71.94 2018 XOne
76.78 2018 3DS
75.39 2018 PSV
nan 2018 Amig
72.0 2018 Wii
72.0 2018 WiiU
71.5 2018 X360
75.0

76.91 2012 PS4
83.0 2012 PSN
76.86 2012 XOne
84.0 2012 NS
65.69 2008 Wii
72.31 2008 All
68.62 2008 X360
67.72 2008 PS3
70.1 2008 PC
53.34 2008 DS
63.54 2008 PS2
71.02 2008 PSP
nan 2008 XB
77.33 2008 3DS
73.0 2008 PSN
79.45 2008 XBL
nan 2008 DC
79.0 2008 PS4
69.1 2010 PC
72.85 2010 All
67.09 2010 X360
65.58 2010 DS
68.18 2010 PS3
68.8 2010 Wii
74.17 2010 PSP
69.06 2010 PS2
85.5 2010 PSV
67.67 2010 3DS
67.67 2010 PS4
nan 2010 VC
67.62 2010 XOne
62.96 2009 Wii
74.47 2009 All
68.5 2009 X360
68.65 2009 PS3
69.98 2009 PC
68.71 2009 PSP
56.85 2009 DS
70.78 2009 PSN
68.3 2009 PS2
84.0 2009 XBL
78.0 2009 PSV
nan 2009 WW
nan 2009 iOS
nan 2009 OSX
81.0 2009 Mob
80.56 1996 GB
74.27 1996 N64
77.9 1996 All
72.36 1996 PS
68.89 1996 PC
82.85 1996 SNES
74.86 1996 SAT
61.08 1996 GEN
61.0 1996 PSP
nan 1996 GBA
70.39 1996 NG
69.71 1996 iOS
nan 1996 PCFX
73.38 1996 Wii
70.31 1989 GB
80.65 1989 NES
70.42 1989 All
74.22 1989 PC
nan 1989 PS
nan 1989 2600
88.41 1989 SNES
nan 1989 GEN
nan 1989 SAT
71.68 2019 Al

# Préparation des échantillons à utiliser en machine learning

In [37]:
#df.to_pickle('vgchartz_games_merge_metacritic_igdb_fillna.pkl')

In [88]:
df = df[df.id_igdb_game.notna()]

In [101]:
df.game_collection.fillna(df.cleaned_name, inplace = True)

In [102]:
df.year = df.year.astype(int)
bool_columns = ['genre_adventure', 'genre_arcade',
       'genre_card-and-board-game', 'genre_fighting',
       'genre_hack-and-slash-beat-em-up', 'genre_indie', 'genre_moba',
       'genre_music', 'genre_pinball', 'genre_platform',
       'genre_point-and-click', 'genre_puzzle', 'genre_quiz-trivia',
       'genre_racing', 'genre_real-time-strategy-rts',
       'genre_role-playing-rpg', 'genre_shooter', 'genre_simulator',
       'genre_sport', 'genre_strategy', 'genre_tactical',
       'genre_turn-based-strategy-tbs', 'genre_visual-novel',
       'mode_battle-royale', 'mode_co-operative', 'mode_mmo',
       'mode_multiplayer', 'mode_single-player', 'mode_split-screen',
       'perspective_auditory', 'perspective_bird-view_isometric',
       'perspective_first-person', 'perspective_side-view', 'perspective_text',
       'perspective_third-person', 'perspective_virtual-reality']

df[bool_columns] = df[bool_columns].astype('bool')

In [104]:
df.to_pickle('vgchartz_games_merge_metacritic_igdb_fillna.pkl')

In [92]:
df_model_1 = df[['total_sales', 'platform', 'release_month', 'year', 'publisher', 'developer',
       'meta_score', 'meta_nb_user_critics', 'meta_user_score',
       'meta_critic_review_positive_perc',
       'meta_critic_review_mixed_perc', 'meta_user_review_positive_perc',
       'meta_user_review_positive', 'meta_user_review_negative',
       'meta_user_review_mixed', 'meta_user_review_mixed_perc',
       'meta_critic_review_negative_perc', 'meta_user_review_negative_perc',
       'game_collection', 'nb_platforms',
       'genre_adventure', 'genre_arcade',
       'genre_card-and-board-game', 'genre_fighting',
       'genre_hack-and-slash-beat-em-up', 'genre_indie', 'genre_moba',
       'genre_music', 'genre_pinball', 'genre_platform',
       'genre_point-and-click', 'genre_puzzle', 'genre_quiz-trivia',
       'genre_racing', 'genre_real-time-strategy-rts',
       'genre_role-playing-rpg', 'genre_shooter', 'genre_simulator',
       'genre_sport', 'genre_strategy', 'genre_tactical',
       'genre_turn-based-strategy-tbs', 'genre_visual-novel',
       'mode_battle-royale', 'mode_co-operative', 'mode_mmo',
       'mode_multiplayer', 'mode_single-player', 'mode_split-screen',
       'perspective_auditory', 'perspective_bird-view_isometric',
       'perspective_first-person', 'perspective_side-view', 'perspective_text',
       'perspective_third-person', 'perspective_virtual-reality',
       'platform_name', 'platform_generation', 'platform_family',
       'platform_category']]

In [93]:
df_model_2 = df[['total_sales', 'platform', 'release_month', 'year', 'publisher', 'developer',
       'meta_score', 'meta_nb_user_critics', 'meta_user_score',
       'game_collection', 'nb_platforms',
       'genre_adventure', 'genre_arcade',
       'genre_card-and-board-game', 'genre_fighting',
       'genre_hack-and-slash-beat-em-up', 'genre_indie', 'genre_moba',
       'genre_music', 'genre_pinball', 'genre_platform',
       'genre_point-and-click', 'genre_puzzle', 'genre_quiz-trivia',
       'genre_racing', 'genre_real-time-strategy-rts',
       'genre_role-playing-rpg', 'genre_shooter', 'genre_simulator',
       'genre_sport', 'genre_strategy', 'genre_tactical',
       'genre_turn-based-strategy-tbs', 'genre_visual-novel',
       'mode_battle-royale', 'mode_co-operative', 'mode_mmo',
       'mode_multiplayer', 'mode_single-player', 'mode_split-screen',
       'perspective_auditory', 'perspective_bird-view_isometric',
       'perspective_first-person', 'perspective_side-view', 'perspective_text',
       'perspective_third-person', 'perspective_virtual-reality',
       'platform_name', 'platform_generation', 'platform_family',
       'platform_category']]

In [94]:
df_model_1 = df_model_1[
    (df_model_1.meta_critic_review_positive_perc.notna()) & (df_model_1.meta_user_review_positive_perc.notna()) & (df_model_1.meta_user_review_positive.notna()) & (df_model_1.meta_user_review_mixed_perc.notna())]

In [96]:
# Sauvegarde des échantillons

df_model_1.to_pickle('df_model_1.pkl')
df_model_2.to_pickle('df_model_2.pkl')